In [ ]:
import src.datasets as datasets
import src.surrogate as surrogate
import src.search as search

In [ ]:
load_data_func = getattr(datasets, f'cement_data')
X_train, X_test, y_train, y_test = load_data_func('./data/concrete_processed.csv')

load_data_loader_func = getattr(datasets, f'lightgbm_load_data')
train_loader, val_loader = load_data_loader_func(X_train, X_test, y_train, y_test)

train_func = getattr(surrogate, f'lightgbm_train')
model = train_func(train_loader, val_loader)

# search_func = getattr(search, f'ga_search')
# search_func(model, train_loader, val_loader)




In [ ]:
from deap import base, creator, tools
import numpy as np
import random

In [ ]:
x_min,x_max = np.min(X_train, axis=0), np.max(X_train, axis=0)

In [ ]:
n_features = X_train.shape[1]

In [ ]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', np.ndarray, fitness=creator.FitnessMax)

In [ ]:
def generate_individual():
    return np.random.uniform(x_min, x_max)


In [ ]:
toolbox = base.Toolbox()
toolbox.register('attr_float', generate_individual)
# min_max 차원이 8개이기에 n을 1로 설정 하면 8개의 변수를 가진 ind 생성!
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)


In [ ]:
predict_func = getattr(surrogate, f'lightgbm_predict')

In [ ]:
gt_y = y_test[0]

In [ ]:
def fitness(population):

    population = np.array(population)
    print('population shape : ', population.shape)
    y_pred = predict_func(model=model, X_test=population)

    fit_fun = -(y_pred - gt_y)**2
    return fit_fun


In [ ]:
toolbox.register('evaluate', fitness)


In [ ]:
population = toolbox.population(n=100)


In [ ]:
ETA_CX = 5.0 # 초기에 작게(탐색), 점점 크게(exploitation)
toolbox.register('mate', tools.cxSimulatedBinary, eta=ETA_CX)

In [ ]:
MUTPB = 0.2          # 돌연변이 적용 확률(전역)
INDPB = 0.1          # 각 변수별 변이 확률
sigma_list = [(ub - lb)/10.0 for (lb,ub) in zip(x_min, x_max)]  # 변수 범위에 따른 sigma 값 

toolbox.register('mutate', tools.mutGaussian, mu=[0.0]*(len(x_min)), sigma=sigma_list, indpb=INDPB)


추후 selNSGA2(), selSPEA2(), selNSGA3() 등 파레토 최적화 알고리즘으로 변경 필요

In [ ]:
toolbox.register('select', tools.selTournament)

In [ ]:
population = toolbox.population(n=100)


In [ ]:
cxpb = 0.5
mutpb = 0.5


In [ ]:
offspring = [toolbox.clone(ind) for ind in population]

In [ ]:
for i in range(1, len(offspring), 2):
    if random.random() < cxpb:
        offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                        offspring[i])
        del offspring[i - 1].fitness.values, offspring[i].fitness.values



In [ ]:
offspring = [np.clip(individual, x_min, x_max) for individual in offspring]


In [ ]:
for i in range(len(offspring)):
    if random.random() < mutpb:
        offspring[i], = toolbox.mutate(offspring[i])
        # del offspring[i].fitness


In [ ]:
offspring = [np.clip(individual, x_min, x_max) for individual in offspring]


In [ ]:
invalid_ind = [ind for ind in population if not ind.fitness.valid]

In [ ]:
fitness_scores = toolbox.evaluate(invalid_ind)
for ind, fit in zip(invalid_ind, fitness_scores):
    ind.fitness.values = (fit,)

In [ ]:
import faiss
def fast_nearest_neighbors(population, k):
    n, d = population.shape
    kmeans = faiss.Kmeans(d, k, niter=20, verbose=False)
    population = population.astype('float32')
    kmeans.train(population)

    cluster_labels = kmeans.index.search(population, 1)[1].flatten()  
    centroids = kmeans.centroids
    
    return cluster_labels, centroids


In [ ]:
cluster_labels, centroids = fast_nearest_neighbors(np.array(population),k=len(population)//10)

In [ ]:
len(population + offspring)

In [ ]:
cluster_labels, centroids = fast_nearest_neighbors(np.array(population + offspring),k=len(population)//10)

In [ ]:
cluster_labels

In [ ]:
i = 1
cluster_labels == i

In [ ]:
cluster_idx = np.where(cluster_labels == i)[0]


In [ ]:
cluster_idx

In [ ]:
k = [(population + offspring)[i] for i in cluster_idx]

In [ ]:
toolbox.select

In [ ]:
len(population + offspring)//2

In [ ]:
len([(population + offspring)[i] for i in cluster_idx])


In [ ]:
len(population)//10

In [ ]:
pop = toolbox.select([(population + offspring)[i] for i in cluster_idx],k=len(population + offspring)//2,tournsize=3)


In [ ]:
# %%time
# offspring = list(map(toolbox.clone, population))


In [ ]:
# selTournamentDCD는 4의 배수여야함 ... 
# offspring = toolbox.select(population, len(population))